# Tweets

In [1]:
import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel

from pprint import pprint

import spacy

import pickle
import re 
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

import matplotlib.pyplot as plt 
import pandas as pd

/usr/local/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
/usr/local/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_least_angle.py:34: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/usr/local/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_least_angle.py:164: DeprecationWarning: 

# Importing Data
Data: https://datapane.com/u/khuyentran1401/reports/Y3Ya6e3/processed-tweets/

If you want to get access to the data above and follow along with the article, download the data and put the data in your current directory, then run:

In [2]:
tweets = pd.read_csv('dp-export-8940.csv') #Change this with the name of your downloaded file
tweets = tweets.Tweets.values.tolist()

# Turn the list of string into a list of tokens
tweets = [t.split(',') for t in tweets]

/usr/local/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# How to use LDA Model
Topic modeling involves counting words and grouping similar word patterns to describe topics within the data. If the model knows the word frequency, and which words often appear in the same document, it will discover patterns that can group different words together.

We start with converting a collection of words to a bag of words, which is a list of tuples (word_id, word_frequency). gensim.corpora.Dictionary is a great tool for this:

In [3]:
id2word = Dictionary(tweets)
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in tweets]
print(corpus[:1])

/usr/local/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[[(0, 1), (1, 1), (2, 1), (3, 3), (4, 1), (5, 2), (6, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 2), (12, 2), (13, 1), (14, 1), (15, 1), (16, 2), (17, 1), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 2), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 2), (35, 1), (36, 2), (37, 1), (38, 9), (39, 1), (40, 1), (41, 1), (42, 2), (43, 2), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 2), (53, 2), (54, 2), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 2), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 2), (71, 1), (72, 5), (73, 2), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 5), (83, 1), (84, 2), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 5), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 1), (105, 4), (106, 1), (107, 1), (108, 1), (109, 1), (110, 1)

## What do these tuples mean? Let’s convert them into human readable format to understand:

In [4]:
[[(id2word[i], freq) for i, freq in doc] for doc in corpus[:1]]

/usr/local/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[[("'d", 1),
  ('-', 1),
  ('absolutely', 1),
  ('aca', 3),
  ('act', 1),
  ('action', 2),
  ('add', 2),
  ('administrative', 1),
  ('affordable', 1),
  ('allow', 1),
  ('amazing', 1),
  ('arrive', 2),
  ('ask', 2),
  ('audits', 1),
  ('av', 1),
  ('avoid', 1),
  ('away', 2),
  ('back', 1),
  ('ball', 1),
  ('baseball', 1),
  ('beget', 2),
  ('begin', 1),
  ('behavior', 1),
  ('believe', 1),
  ('bid', 1),
  ('big', 2),
  ('billy', 1),
  ('board', 1),
  ('bout', 1),
  ('branch', 1),
  ('break', 1),
  ('bring', 1),
  ('brother', 1),
  ('build', 1),
  ('call', 2),
  ('can', 1),
  ('cap', 2),
  ('car', 1),
  ('care', 9),
  ('cell', 1),
  ('certainly', 1),
  ('chair', 1),
  ('change', 2),
  ('chant', 2),
  ('chicken', 1),
  ('child', 1),
  ('chip', 1),
  ('choice', 1),
  ('choke', 1),
  ('chuck', 1),
  ('classification', 1),
  ('close', 1),
  ('come', 2),
  ('community', 2),
  ('compare', 2),
  ('competition', 1),
  ('competitively', 1),
  ('compliant', 1),
  ('conclusion', 1),
  ('conferen

## Now let’s build an LDA topic model. We will use gensim.models.ldamodel.LdaModel for this:

In [5]:
# Build LDA model
lda_model = LdaModel(corpus=corpus,
                   id2word=id2word,
                   num_topics=10, 
                   random_state=0,
                   chunksize=100,
                   alpha='auto',
                   per_word_topics=True)

pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

/usr/local/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.144*"video" + 0.127*"like" + 0.098*"follow" + 0.082*"check" + '
  '0.045*"people" + 0.042*"automatically" + 0.029*"unfollowed" + '
  '0.024*"follower" + 0.021*"person" + 0.016*"new"'),
 (1,
  '0.040*"game" + 0.033*"play" + 0.027*"win" + 0.023*"team" + 0.018*"player" + '
  '0.015*"good" + 0.014*"great" + 0.013*"half" + 0.012*"season" + 0.011*"fan"'),
 (2,
  '0.017*"go" + 0.015*"good" + 0.013*"love" + 0.011*"think" + 0.011*"know" + '
  '0.010*"watch" + 0.010*"time" + 0.008*"look" + 0.008*"come" + 0.007*"year"'),
 (3,
  '0.140*"more" + 0.050*"today" + 0.017*"cancer" + 0.015*"pisce" + '
  '0.011*"capricorn" + 0.011*"aquarius" + 0.009*"arie" + 0.007*"feel" + '
  '0.006*"day" + 0.005*"gemini"'),
 (4,
  '0.000*"follow" + 0.000*"thx" + 0.000*"people" + 0.000*"twitt" + '
  '0.000*"trump" + 0.000*"know" + 0.000*"time" + 0.000*"thank" + 0.000*"go" + '
  '0.000*"need"'),
 (5,
  '0.034*"trump" + 0.014*"people" + 0.011*"vote" + 0.011*"think" + '
  '0.010*"know" + 0.008*"country" + 0.008*"n

What are the patterns we can see?
0. Following
1. Gameplay
2. Good time?
3. Astrology
4. Thanks for following
5. Voting
6. Business
7. New year
8. Not sure
9. World leaders ?

In [6]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=tweets, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

/usr/local/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Coherence Score:  0.3577702217149491


# pyLDAvis

In [9]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)

/usr/local/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/anaconda3/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.179362  0.008099       1        1  30.531812
2     -0.185495 -0.109000       2        1  25.034749
9     -0.163385  0.042416       3        1  21.353508
5     -0.146006 -0.115867       4        1   9.387663
6     -0.001287  0.271091       5        1   4.020172
1     -0.048241 -0.174229       6        1   3.485147
8     -0.025679  0.153057       7        1   2.883565
3      0.029057  0.017810       8        1   2.609912
0      0.375638 -0.039021       9        1   0.689848
4      0.344759 -0.054354      10        1   0.003623, topic_info=       Term          Freq         Total Category  logprob  loglift
231  people  16196.000000  16196.000000  Default  30.0000  30.0000
200    more  10278.000000  10278.000000  Default  29.0000  29.0000
627   today  16179.000000  16179.000000  Default  28.0000  28.0000
135   great  16740.000000  16740.000000  Default  27.0000  27.0000
929   video   5546.000000   5546.000000  Default  26.0000  26.0000
..      ...           ...           ...      ...      ...      ...
374    work      0.001492  14281.831748  Topic10 -11.1322  -5.8485
309      so      0.001492   8166.575567  Topic10 -11.1324  -5.2897
819   watch      0.001492  10287.421313  Topic10 -11.1325  -5.5206
808    tell      0.001492   6498.229263  Topic10 -11.1328  -5.0616
72      day      0.001491  18352.901112  Topic10 -11.1331  -6.1001

[814 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
11031      3  0.995126        ):
1          1  0.165846         -
1          2  0.026293         -
1          3  0.644574         -
1          4  0.106384         -
...      ...       ...       ...
2672       2  0.012372      yoga
11399      9  0.972940  youtuber
9344       6  0.990660       zip
3684       1  0.997681         °
12961      4  0.997597         •

[2227 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 3, 10, 6, 7, 2, 9, 4, 1, 5])

**Closing Notes**

Machine learning has become increasingly popular over the past decade, and recent advances in computational availability have led to exponential growth to people looking for ways how new methods can be incorporated to advance the field of Natural Language Processing.

Often, we treat topic models as black-box algorithms, but hopefully, this article addressed to shed light on the underlying math, and intuitions behind it, and high-level code to get you started with any textual data.

In the next article, we’ll go one step deeper into understanding how you can evaluate the performance of topic models, tune its hyper-parameters to get more intuitive and reliable results.

